In [12]:
import leveldb
db_item = leveldb.LevelDB('./taobao-item')

In [13]:
for k in db_item.RangeIter(include_value = False):
    print db_item.Get(k)

{"nick": "慧诺家居专营店", "name": "香薰包衣厨衣柜香包汽车抽屉香熏包熏衣草檀香香囊熏香包", "area": "金华", "query": "ECOONER 熏衣草", "price": 29.9, "_id": 14110927956}
{"nick": "xlfwrs", "name": "德国贺本清小甘菊经典洋甘菊护手霜20ML铁盒/玫瑰/熏衣草", "area": "深圳", "query": "ECOONER 熏衣草", "price": 22.0, "_id": 15529594823}
{"nick": "zhaosanwang", "name": "中药材薰衣草花茶 干花熏衣草500克  55元", "area": "保定", "query": "ECOONER 熏衣草", "price": 55.0, "_id": 16198679866}
{"nick": "伊犁兄弟", "name": "赖希薰衣草干花粒散装填充香包枕头助眠新疆熏衣草精油香料包邮", "area": "伊犁", "query": "ECOONER 熏衣草", "price": 17.5, "_id": 16644289575}
{"nick": "小芬乐易购", "name": "蔬菜普罗旺斯熏衣草种子 植物香草阳台盆栽1克约1000粒薰衣草直销", "area": "北京", "query": "ECOONER 熏衣草", "price": 10.69, "_id": 17284603683}
{"nick": "stone丹丹", "name": "【摆设.摩登东方】 织锦缎/天然熏衣草眼罩/美容觉", "area": "北京", "query": "ECOONER 熏衣草", "price": 49.0, "_id": 17462932109}
{"nick": "stone丹丹", "name": "【摆设.摩登东方】 织锦缎/天然熏衣草眼罩/美容觉", "area": "北京", "query": "ECOONER 熏衣草", "price": 49.0, "_id": 17462948087}
{"nick": "花仙子家居旗舰店", "name": "花仙子衣物香氛袋卧室房间除味香薰随身薰衣", "area": "上海", "query": 

In [3]:
import time
import threading

def demo(n):
    while n >= 0:
        print("n:", n)
        n -= 1
        time.sleep(2)
        print("stop n:", n)

In [10]:
t = threading.Thread(target=demo, args=(10,))
t.start()
t.join()
print("exit")

('n:', 10)
('stop n:', 9)
('n:', 9)
('stop n:', 8)
('n:', 8)
('stop n:', 7)
('n:', 7)
('stop n:', 6)
('n:', 6)
('stop n:', 5)
('n:', 5)
('stop n:', 4)
('n:', 4)
('stop n:', 3)
('n:', 3)
('stop n:', 2)
('n:', 2)
('stop n:', 1)
('n:', 1)
('stop n:', 0)
('n:', 0)
('stop n:', -1)
exit
